Training a 2D bbox to segment model
---

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from evaluation.eval_utils import get_seg_bbox
from pathlib import Path
from tqdm import tqdm

import SimpleITK as sitk

In [ ]:
data_folder = Path("/media/liushifeng/KINGSTON/ULS Jan 2025/ULS23/novel_data/ULS23_DeepLesion3D")
seg_folder = data_folder / "labels"
ct_folder = data_folder / "images"

with open(data_folder / "train.txt", "r") as f:
    train_names = [x.strip() for x in f.readlines()]
with open(data_folder / "val.txt", "r") as f:
    val_names = [x.strip() for x in f.readlines()]

In [ ]:
# get input crops and segments from GT 3D segs

In [ ]:
filenames = [x for x in os.listdir(seg_folder) if ".zip" not in x]
seg_paths = {x: seg_folder / x for x in filenames}
ct_paths = {x: ct_folder / x for x in filenames}

In [ ]:
ignored_train_samples = [x.strip() for x in
    """
    000148_04_01_034_lesion_01
    003287_01_01_188_lesion_01
    003931_01_01_078_lesion_01
    000026_06_01_257_lesion_01
    000346_01_01_085_lesion_01
    000215_05_01_096_lesion_01
    001354_04_02_305_lesion_01
    001564_02_02_513_lesion_01
    """.split("\n") if x.strip()]

In [ ]:
from utils.plot import transparent_cmap

qc_filenames = [x.strip() for x in
    """
001564_02_02_513_lesion_01
    """.split("\n") if x.strip()]

# for filename in list(seg_paths.keys()):
for filename in qc_filenames:
    if filename.split(".")[0] in ignored_train_samples:
        print("skipped:", filename.split(".")[0])
        continue
    filename += ".nii.gz"

    ct = sitk.ReadImage(ct_paths[filename])
    ct_array = sitk.GetArrayFromImage(ct)
    seg = sitk.ReadImage(seg_paths[filename])
    seg_array = sitk.GetArrayFromImage(seg)
    print(filename.split(".")[0])
    print("array sizes", ct_array.shape, seg_array.shape)

    seg_slice_indices = np.where(seg_array.any(axis=(1,2)) > 0)[0]

    # for i in seg_slice_indices:
    #     print(i)
    mid_slice_index = seg_slice_indices[len(seg_slice_indices) // 2]
    for i in [mid_slice_index]:
        ct_slice = ct_array[i]
        seg_slice = seg_array[i]
        crop_bbox = get_seg_bbox(seg_slice)

        print("slice id", seg_slice_indices)
        print("bbox", crop_bbox)

        # crops areas for plotting
        m_limit = min(min(crop_bbox[:2]), ct_slice.shape[1] - crop_bbox[2], ct_slice.shape[0] - crop_bbox[3])
        m = min(50, m_limit) - 2
        crop_bbox[0] -= m
        crop_bbox[1] -= m
        crop_bbox[2] += m
        crop_bbox[3] += m
        ct_slice_crop = ct_slice[crop_bbox[1]:crop_bbox[3]+1, crop_bbox[0]:crop_bbox[2]+1]
        seg_slice_crop = seg_slice[crop_bbox[1]:crop_bbox[3]+1, crop_bbox[0]:crop_bbox[2]+1]
        _, axes = plt.subplots(1, 2, figsize=(10, 5))

        vmin, vmax = 200, 700
        axes[0].imshow(ct_slice_crop,
                       vmin=vmin, vmax=vmax,
                       cmap='gray')
        axes[1].imshow(ct_slice_crop,
                       vmin=vmin, vmax=vmax,
                       cmap='gray')
        axes[1].imshow(seg_slice_crop, cmap=transparent_cmap("blue"), alpha=0.4)
        for ax in axes:
            ax.axis("off")
        plt.show()


        # plot the entire area
        _, axes = plt.subplots(1, 2, figsize=(10, 5))
        axes[0].imshow(ct_slice,
                       # vmin=-500, vmax=500,
                       cmap='gray')
        axes[1].imshow(ct_slice,
                       # vmin=-500, vmax=500,
                       cmap='gray')
        axes[1].imshow(seg_slice, cmap=transparent_cmap("blue"), alpha=0.4)

        for ax in axes:
            ax.axis("off")
        plt.show()

In [ ]:
plt.imshow(ct_slice, cmap='gray'); plt.axis("off");
plt.imshow(seg_slice, cmap='jet', alpha=0.5); plt.axis("off");

In [ ]:

results = []
for pred in tqdm(os.listdir(pred_folder)):
    lesion_name = pred.replace("_pred.nrrd", "")
    gt_path = gt_folder / f"{lesion_name}.nii.gz.zip"